## Imports

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from model import Neuro_gambit

## ELO init

In [ ]:
elo = 2500

## Device init

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # using cuda
cpu = torch.device('cpu') # using cpu

## Loading Tensors

In [ ]:
X = torch.load('./data/X_tensor_'+str(elo)+'.pt').to(device)
Y = torch.load('./data/Y_tensor_'+str(elo)+'.pt').to(device)
print(X.shape)
print(Y.shape)

# seperating the Y
Y1 = Y[:, :8]
Y2 = Y[:, 8:16]
Y3 = Y[:, 16:24]
Y4 = Y[:, 24:32]
Y5 = Y[:, 32:]

Y = [Y1,Y2,Y3,Y4,Y5]

## Model class init

In [ ]:
model = Neuro_gambit().to(device)

# epochs, loss, and optim
learning_rate = 0.01
n_epochs = 1000000

# loss and optimizer functions from pytorch
criterion = nn.MSELoss() # MSE function
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate) # stochastic gradient descent function

## Training

In [ ]:
for epoch in range(n_epochs):
    # forward
    y_preds = model(X) # will output a tuple of 5 tensors

    total_loss = 0
    for i in range(len(y_preds)): # calculating the loss per tensor
        y_pred = y_preds[i]
        total_loss += criterion(y_pred, Y[i])

    # backward
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {total_loss.item():.4f}', end='\r')

## Saving the model

In [ ]:
# Save the model
torch.save(model.state_dict(), './models/neuro_gambit_'+str(elo)+'.pt')

## Loading saved model

In [ ]:
model.load_state_dict(torch.load('./models/neuro_gambit_'+str(elo)+'.pt')) # it takes the loaded dictionary, not the path file itself
model.eval()

## Sample test

In [13]:
import torch.nn.functional as F

import chess
from chess_transformation import linear_to_board, matrix_to_board, full_move_to_algebraic

def get_all_possible_moves(board : list[str], player : str):
    board : chess.Board = matrix_to_board(linear_to_board(board), player)
    print(board)
    possible_moves = []
    player_col = chess.WHITE if player == 'white' else chess.BLACK
    for move in board.legal_moves:
        if board.turn == player_col:
            uci_move = move.uci()
            possible_moves.append(uci_move)

    return possible_moves

def get_move_probability(move : str, tensor_tuple : tuple[torch.Tensor]):
    pos_rank_labels = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
    pos_file_labels = {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7}
    promotion_labels = {'q': 0, 'r': 1, 'b': 2, 'n': 3}

    # restruct tensors to list matrix
    tensor_matrix = []
    for tensor in tensor_tuple:
        tensor_list = F.softmax(tensor, dim=0).to(cpu).tolist()
        tensor_matrix.append(tensor_list)

    # get indices
    o_r_i = pos_rank_labels[move[0]]
    o_f_i = pos_file_labels[move[1]]
    d_r_i = pos_rank_labels[move[2]]
    d_f_i = pos_file_labels[move[3]]

    # get probabilities
    p_o_r = tensor_matrix[0][o_r_i]
    p_o_f = tensor_matrix[1][o_f_i]
    p_d_r = tensor_matrix[2][d_r_i]
    p_d_f = tensor_matrix[3][d_f_i]

    return p_o_r*p_o_f*p_d_r*p_d_f

data_pandas = pd.read_csv('./data/games_cleaned_'+str(elo)+'_scrambled.csv')

# Testing
game = 506
data_pandas = pd.read_csv('./data/games_cleaned_'+str(elo)+'_scrambled.csv')
position_columns = ['p'+str(i) for i in range(64)]
board_linear = data_pandas.iloc[game][position_columns].to_list()
player = data_pandas.iloc[game]['player']
# player = 'black'
best = data_pandas.iloc[game]['uci']
print('player:', player)
print('best move:', best)


possible_moves = get_all_possible_moves(board_linear, player)

# for i in range(len(possible_moves)):
#     print(possible_moves[i], end='\n' if i%5==0 else ',')

print('prediction:')
with torch.no_grad():
    x = X[0].view(1,-1)
    y = Y[0][0],Y[1][0],Y[2][0],Y[3][0],Y[4][0]
    y_preds = [t.view(-1,) for t in model(x)]
    print(y_preds)
    for y_iter in y:
        print(y_iter)
    for y_pred in y_preds:
        print(F.softmax(y_pred, dim=0))

    possible_moves_probabilities = {move : get_move_probability(move, y_preds) for move in possible_moves}
    print('prob of best move:', possible_moves_probabilities[best])
    max_key_val = {'move' : '', 'prob' : 0}
    for key in possible_moves_probabilities.keys():
        value = possible_moves_probabilities[key]
        if value > max_key_val['prob']:
            max_key_val['move'] = key
            max_key_val['prob'] = value
    print(max_key_val)


player: white
best move: e5g7
. . . q n r k .
. r . . p p b p
p N p . . . p .
. . P p B . . .
. . . Q . . b .
. P . . P . P .
P . . . . P B P
R . . . . R K .
prediction:
[tensor([-2.5979e-02,  3.8682e-02,  1.0348e+00, -2.9506e-02,  9.5955e-03,
         2.5462e-02, -2.0862e-05, -5.1327e-02], device='cuda:0'), tensor([ 0.0109,  0.9911, -0.0153, -0.0138, -0.0247,  0.0106, -0.0286,  0.0630],
       device='cuda:0'), tensor([-0.0052,  0.0264,  0.9968,  0.0011,  0.0226, -0.0144, -0.0529,  0.0482],
       device='cuda:0'), tensor([ 2.4102e-02,  7.7048e-04,  1.0114e+00, -2.4847e-02, -3.4215e-02,
         2.4004e-02,  1.9166e-02, -1.7331e-02], device='cuda:0'), tensor([ 0.0152, -0.0032,  0.0005,  0.0030], device='cuda:0')]
tensor([0., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 1., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0.], device='cuda:0')
te